In [ ]:
%pip install "calcbench-api-client[Pandas]"

In [2]:
import pyarrow.parquet as pq
import pyarrow.compute as pc
import pyarrow as pa
import numpy as np
import calcbench as cb
from ipydatagrid import DataGrid
from qgridnext import show_grid
from scipy.stats import zscore
from datetime import date

### File is the Calcbench Point-In-Time data, see https://github.com/calcbench/notebooks/blob/master/standardized_numeric_point_in_time.ipynb
### Talk to us@calcbench.com if you want a historical file.

In [3]:
# last date
last_date = pd.to_datetime(date(2024, 9, 30))

# XBRL data starts ~2010
first_date = pd.to_datetime(date(2010, 1, 1))

In [4]:
d = pq.read_table("C:/Users/andre/Downloads/standardized_data_oct_23.parquet")

In [6]:
expenses = [
    "CostOfRevenue",
    "SGAExpense",
    "InterestExpense",
    "IncomeTaxes",
]

expr = pc.field("metric").isin(["Revenue", "NetIncome"] + expenses)

In [7]:
metric_data = d.filter(expr).to_pandas()

In [8]:
metric_data

value  \
ticker     metric        fiscal_period date_reported                           
0000739708 CostOfRevenue 2010-2        2011-08-03 17:22:45.000  6.009160e+08   
                         2010-3        2011-10-31 16:01:01.000  5.790980e+08   
                         2010-4        2011-10-31 16:01:01.000  2.491906e+09   
                         2011-1        2012-05-04 08:05:45.000  5.840690e+08   
                         2011-2        2011-08-03 17:22:45.000  6.423400e+08   
...                                                                      ...   
talo       SGAExpense    2023-2        2023-08-08 17:01:38.907  3.318200e+07   
                         2023-3        2023-11-06 17:36:40.703  2.488800e+07   
                         2023-4        2024-02-28 19:30:12.550  3.723600e+07   
                         2024-1        2024-05-06 16:17:24.340  6.984100e+07   
                         2024-2        2024-08-07 16:48:31.517  4.824700e+07   

                                                                revision_number  \
ticker     metric        fiscal_period date_reported                              
0000739708 CostOfRevenue 2010-2        2011-08-03 17:22:45.000                0   
                         2010-3        2011-10-31 16:01:01.000                0   
                         2010-4        2011-10-31 16:01:01.000                0   
                         2011-1        2012-05-04 08:05:45.000                0   
                         2011-2        2011-08-03 17:22:45.000                0   
...                                                                         ...   
talo       SGAExpense    2023-2        2023-08-08 17:01:38.907                0   
                         2023-3        2023-11-06 17:36:40.703                0   
                         2023-4        2024-02-28 19:30:12.550                0   
                         2024-1        2024-05-06 16:17:24.340                0   
                         2024-2        2024-08-07 16:48:31.517                0   

                                                                preliminary  \
ticker     metric        fiscal_period date_reported                          
0000739708 CostOfRevenue 2010-2        2011-08-03 17:22:45.000        False   
                         2010-3        2011-10-31 16:01:01.000        False   
                         2010-4        2011-10-31 16:01:01.000        False   
                         2011-1        2012-05-04 08:05:45.000        False   
                         2011-2        2011-08-03 17:22:45.000        False   
...                                                                     ...   
talo       SGAExpense    2023-2        2023-08-08 17:01:38.907         True   
                         2023-3        2023-11-06 17:36:40.703         True   
                         2023-4        2024-02-28 19:30:12.550         True   
                         2024-1        2024-05-06 16:17:24.340         True   
                         2024-2        2024-08-07 16:48:31.517         True   

                                                                XBRL  \
ticker     metric        fiscal_period date_reported                   
0000739708 CostOfRevenue 2010-2        2011-08-03 17:22:45.000  True   
                         2010-3        2011-10-31 16:01:01.000  True   
                         2010-4        2011-10-31 16:01:01.000  True   
                         2011-1        2012-05-04 08:05:45.000  True   
                         2011-2        2011-08-03 17:22:45.000  True   
...                                                              ...   
talo       SGAExpense    2023-2        2023-08-08 17:01:38.907  True   
                         2023-3        2023-11-06 17:36:40.703  True   
                         2023-4        2024-02-28 19:30:12.550  True   
                         2024-1        2024-05-06 16:17:24.340  True   
                         2024-2        2024-08-07 16:48:31

In [9]:
#do things in millions for readability

metric_data["value"] = metric_data["value"].astype(float) / 1e6

In [ ]:
# Remove small companies

#revenue = revenue[revenue["value"] > 1e7]

In [11]:
# Quarterly only
metric_data = metric_data[~metric_data.index.get_level_values("fiscal_period").str.endswith("-0")]

## Add SIC Code meta-data

In [12]:
company_data = cb.companies(entire_universe=True)

In [13]:
merged = metric_data.reset_index().merge(
    company_data[["ticker", "sic_code"]], right_on=["ticker"], left_on=["ticker"]
)[["ticker", "period_end", "sic_code", "value", 'metric']]

##### Split SIC code into levels, GICs codes would probably be better but Calcbench does not have them
###### https://www.sec.gov/corpfin/division-of-corporation-finance-standard-industrial-classification-sic-code-list

In [14]:
merged["top_level_SIC_code"] = merged["sic_code"].floordiv(1000).astype("string")
merged["second_level_SIC_code"] = (
    (merged["sic_code"] % 1000).floordiv(100).astype("string")
)
merged["third_level_SIC_code"] = (
    (merged["sic_code"] % 100).floordiv(10).astype("string")
)
merged["fourth_level_SIC_code"] = (merged["sic_code"] % 10).astype("string")

In [15]:
merged["ds"] = (merged["period_end"] + pd.offsets.QuarterEnd()).dt.normalize()

In [16]:
merged = merged[
    ~merged.isnull().T.any()
]  # get rid of rows with any null values, the model builder does not like nulls

In [ ]:
# only get retail companies 
#merged = merged[merged['top_level_SIC_code'] == '5']

In [18]:
# Get first record, there are revisions in the Calcbench data set.
merged = merged.groupby(["ticker", "ds", "metric"]).first()

In [19]:
# only want companies for which we have more than a year of history
merged = merged.groupby("ticker").filter(lambda g: g.shape[0] > 4)

In [20]:
# throw out outliers
z_scores = merged.groupby(["ticker", "metric"])["value"].transform(lambda x: zscore(x))
merged = merged[z_scores < 4]

In [21]:
accounts = merged.reset_index().set_index(
    [
        "top_level_SIC_code",
        "second_level_SIC_code",
        "third_level_SIC_code",
        "fourth_level_SIC_code",
        "ticker",
        "ds",
        "metric",
    ]
)["value"].unstack("metric")

In [ ]:
calculated_net_income = accounts["Revenue"] - accounts[expenses].sum(axis=1)

In [ ]:
accounts["Other"] = accounts["NetIncome"] - calculated_net_income

In [ ]:
merged = accounts.reset_index()

In [ ]:
merged = merged[merged['ds'] <= last_date]
merged =merged[ merged['ds'] >= first_date]

In [ ]:
# remove companies for which we do not have current data
merged = merged.groupby("ticker").filter(lambda x: x["ds"].max() == last_date)

In [ ]:
# Get rid of companies for which we do not have enough data
MINIMUM_NUMBER_OF_OBSERVATIONS = 40
merged = merged.groupby("ticker").filter(
    lambda x: x.shape[0] > MINIMUM_NUMBER_OF_OBSERVATIONS
)

In [52]:
# Do not want NetIncome because it is the sum

merged = merged[merged.index.get_level_values('metric') != "NetIncome"]

In [62]:
merged = merged.rename({'value':'y'}, axis=1)

In [63]:
merged.reset_index().to_parquet("test_train_data.parquet", index=False)

In [64]:
merged.shape

(1510994, 7)